In [1]:
!mkdir -p dsaquest

In [3]:
%%writefile dsa_data.py
TOTAL_PROBLEMS = 450

BADGES = {
    "rookie": {"name": "🌱 DSA Rookie"},
    "bronze": {"name": "🥉 Bronze Coder"},
    "silver": {"name": "🥈 Silver Solver"},
    "gold": {"name": "🥇 Gold Master"},
}

STRIVER_SHEET = {
    "Step 1": {
        "name": "Learn the Basics",
        "description": "Things to know before starting DSA",
        "icon": "📚",
        "xp": 10,
        "problems": [
            {
                "id": "p1",
                "title": "User Input / Output",
                "difficulty": "Easy",
                "concepts": ["Basic I/O"],
                "link": "https://takeuforward.org/"
            },
            {
                "id": "p2",
                "title": "Data Types",
                "difficulty": "Easy",
                "concepts": ["Data Types"],
                "link": "https://takeuforward.org/"
            }
        ]
    },
    "Step 2": {
        "name": "Sorting Techniques",
        "description": "Learn basic sorting algorithms",
        "icon": "🔢",
        "xp": 20,
        "problems": [
            {
                "id": "p3",
                "title": "Selection Sort",
                "difficulty": "Medium",
                "concepts": ["Sorting", "Arrays"],
                "link": "https://takeuforward.org/"
            },
            {
                "id": "p4",
                "title": "Bubble Sort",
                "difficulty": "Medium",
                "concepts": ["Sorting", "Arrays"],
                "link": "https://takeuforward.org/"
            }
        ]
    },
    "Step 3": {
        "name": "Arrays",
        "description": "Solve array problems",
        "icon": "🧊",
        "xp": 30,
        "problems": [
            {
                "id": "p5",
                "title": "Two Sum",
                "difficulty": "Easy",
                "concepts": ["Arrays", "Hashing"],
                "link": "https://takeuforward.org/"
            }
        ]
    }
}

Overwriting dsa_data.py


In [4]:
%%writefile backend_user.py
import json
import os

USER_FILE = "users.json"
CURRENT_USER_FILE = "current_user.json"

class AuthResponse:
    def __init__(self, error=None, data=None):
        self.error = error
        self.data = data

class AuthData:
    def __init__(self, user=None):
        self.user = user

def _load_users():
    if not os.path.exists(USER_FILE):
        return {}
    try:
        with open(USER_FILE, "r") as f:
            return json.load(f)
    except:
        return {}

def _save_users(users):
    with open(USER_FILE, "w") as f:
        json.dump(users, f)

def _set_current_user(user):
    with open(CURRENT_USER_FILE, "w") as f:
        json.dump(user, f)

def get_current_user():
    if not os.path.exists(CURRENT_USER_FILE):
        return None
    try:
        with open(CURRENT_USER_FILE, "r") as f:
            return json.load(f)
    except:
        return None

def sign_in(email, password):
    users = _load_users()
    if email not in users:
        return AuthResponse(error="User not found")
    
    if users[email]["password"] != password:
        return AuthResponse(error="Invalid password")
    
    user_data = {"email": email, "username": users[email].get("username", email)}
    _set_current_user(user_data)
    return AuthResponse(data=AuthData(user=user_data))

def sign_up(email, password, username=None):
    users = _load_users()
    if email in users:
        return AuthResponse(error="User already exists")
    
    users[email] = {
        "password": password,
        "username": username or email
    }
    _save_users(users)
    return AuthResponse(data=AuthData(user={"email": email, "username": username}))

def sign_out():
    if os.path.exists(CURRENT_USER_FILE):
        os.remove(CURRENT_USER_FILE)

Writing backend_user.py


In [5]:
%%writefile progress_tracker.py
import json
import os
from backend_user import get_current_user
from dsa_data import TOTAL_PROBLEMS, STRIVER_SHEET

TRACKER_FILE = "progress.json"

class ProgressTracker:
    def __init__(self, user_email):
        self.user_email = user_email
        self.data = self._load_data()

    def _load_data(self):
        if not os.path.exists(TRACKER_FILE):
            return {}
        try:
            with open(TRACKER_FILE, "r") as f:
                all_data = json.load(f)
                user_data = all_data.get(self.user_email, {})
                defaults = {
                    "badges_earned": [],
                    "total_xp": 0,
                    "current_streak": 0,
                    "completed_problems": []
                }
                for k, v in defaults.items():
                    if k not in user_data:
                        user_data[k] = v
                return user_data
        except:
            return {
                "badges_earned": [],
                "total_xp": 0,
                "current_streak": 0,
                "completed_problems": []
            }

    def _save_data(self):
        all_data = {}
        if os.path.exists(TRACKER_FILE):
            try:
                with open(TRACKER_FILE, "r") as f:
                    all_data = json.load(f)
            except:
                pass
        
        all_data[self.user_email] = self.data
        with open(TRACKER_FILE, "w") as f:
            json.dump(all_data, f)

    @property
    def badges_earned(self):
        return self.data.get("badges_earned", [])

    @property
    def total_xp(self):
        return self.data.get("total_xp", 0)

    @total_xp.setter
    def total_xp(self, value):
        self.data["total_xp"] = value
        self._save_data()

    @property
    def current_streak(self):
        return self.data.get("current_streak", 0)

    @property
    def completed_problems(self):
        return self.data.get("completed_problems", [])

    def get_progress_percentage(self):
        if TOTAL_PROBLEMS == 0:
            return 0
        return (len(self.completed_problems) / TOTAL_PROBLEMS) * 100

    def get_step_progress(self, step_key):
        step = STRIVER_SHEET.get(step_key)
        if not step:
            return 0
        problems = step.get("problems", [])
        if not problems:
            return 0
        
        completed_in_step = [p for p in problems if p["id"] in self.completed_problems]
        return (len(completed_in_step) / len(problems)) * 100

    def mark_problem_complete(self, problem_id, xp_reward):
        if problem_id in self.completed_problems:
            return False, []
        
        self.data["completed_problems"].append(problem_id)
        self.data["total_xp"] += xp_reward
        
        # Simple badge logic
        new_badges = []
        if len(self.completed_problems) >= 1 and "rookie" not in self.badges_earned:
            self.data["badges_earned"].append("rookie")
            new_badges.append("DSA Rookie")
        
        self._save_data()
        return True, new_badges

    def add_xp_from_game(self, xp):
        self.data["total_xp"] += xp
        self._save_data()

    def save(self):
        self._save_data()

def get_progress_tracker():
    user = get_current_user()
    if not user:
        return None
    return ProgressTracker(user["email"])


Writing progress_tracker.py


In [6]:
%%writefile games.py
import random
from typing import Dict, Any, List

REARRANGE_QUESTIONS = [
    {
        "id": "rearr_1",
        "title": "Print all elements of an array",
        "language": "Python",
        "lines_correct": [
            "arr = [1, 2, 3, 4]",
            "for x in arr:",
            "    print(x)",
        ],
    },
    {
        "id": "rearr_2",
        "title": "Compute sum of an array",
        "language": "C++",
        "lines_correct": [
            "int sum = 0;",
            "for (int x : arr) {",
            "    sum += x;",
            "}",
        ],
    },
]

QUIZ_QUESTIONS = [
    {
        "id": "quiz_1",
        "question": "Which data structure gives O(1) average time for search, insert, and delete?",
        "options": ["Array", "Linked List", "Hash Map", "Binary Search Tree"],
        "answer_index": 2,
        "explanation": "Hash maps (or unordered maps) provide average O(1) for search, insert and delete.",
    },
    {
        "id": "quiz_2",
        "question": "What is the time complexity of binary search?",
        "options": ["O(n)", "O(log n)", "O(n log n)", "O(1)"],
        "answer_index": 1,
        "explanation": "Binary search repeatedly halves the search space, so it is O(log n).",
    },
    {
        "id": "quiz_3",
        "question": "Which traversal visits nodes in sorted order in a BST?",
        "options": ["Preorder", "Inorder", "Postorder", "Level order"],
        "answer_index": 1,
        "explanation": "Inorder traversal of a Binary Search Tree yields nodes in sorted order.",
    },
]

FILL_BLANK_QUESTIONS = [
    {
        "id": "fill_1",
        "language": "Python",
        "snippet": "for i in range(___):\\n    print(i)",
        "answer": "n",
        "hint": "We usually loop from 0 to n-1.",
    },
    {
        "id": "fill_2",
        "language": "C++",
        "snippet": "for (int i = 0; i < ___; i++) {\\n    cout << i << endl;\\n}",
        "answer": "n",
        "hint": "Classic C++ for-loop from 0 to n-1.",
    },
    {
        "id": "fill_3",
        "language": "Python",
        "snippet": "if left <= right:\\n    mid = (left + right) // ___",
        "answer": "2",
        "hint": "Binary search splits the subarray in half.",
    },
]

def get_rearrange_question(idx):
    q = REARRANGE_QUESTIONS[idx % len(REARRANGE_QUESTIONS)]
    shuffled = list(enumerate(q["lines_correct"], start=1))
    random.seed(q["id"])
    random.shuffle(shuffled)
    return q, shuffled

def check_rearrange_answer(q, user_indices):
    shuffled = list(enumerate(q["lines_correct"], start=1))
    random.seed(q["id"])
    random.shuffle(shuffled)
    
    user_lines = []
    for idx in user_indices:
        found = False
        for j, line in shuffled:
            if j == idx:
                user_lines.append(line)
                found = True
                break
        if not found:
            return False 
            
    return user_lines == q["lines_correct"]

def get_quiz_question(idx):
    return QUIZ_QUESTIONS[idx % len(QUIZ_QUESTIONS)]

def get_fill_blank_question(idx):
    return FILL_BLANK_QUESTIONS[idx % len(FILL_BLANK_QUESTIONS)]


Writing games.py


In [7]:
%%writefile dsaquest/__init__.py
# empty


Writing dsaquest/__init__.py


In [8]:
%%writefile dsaquest/main_dsa_agents.py


# ============================================================
# dsaquest/main_dsa_agents.py
# ============================================================

import os
import json
import asyncio
import traceback
from typing import Dict, Any

# Mock ADK if not installed (for local agent testing/safety)
try:
    from google.adk.agents import Agent, SequentialAgent, ParallelAgent
    from google.adk.runners import InMemoryRunner
    from google.adk.models.google_llm import Gemini
    HAS_ADK = True
except ImportError:
    HAS_ADK = False
    print("WARNING: google.adk not found. Using mock implementation.")
    
    # Minimal Mock Classes
    class Gemini:
        def __init__(self, model=None, retry_options=None): pass
    
    class Agent:
        def __init__(self, name, model, instruction, output_key):
            self.name = name
            self.output_key = output_key
    
    class ParallelAgent:
        def __init__(self, name, sub_agents): pass
        
    class SequentialAgent:
        def __init__(self, name, sub_agents): pass
        
    class InMemoryRunner:
        def __init__(self, agent): pass
        async def run_debug(self, user_input):
            return {
                "final_output": {
                    "story": "This is a mock story about the problem.",
                    "analogy": "This is a mock analogy.",
                    "learning_tip": "This is a mock tip.",
                    "code": {
                        "python": "def solution(): pass",
                        "java": "class Solution {}",
                        "c": "void solution() {}",
                        "cpp": "class Solution {};"
                    }
                }
            }

# Load Gemini API key from Kaggle Secrets if available
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Gemini API key loaded.")
except Exception:
    pass 

# Aggressive retry config to handle rate limits
retry_config = dict(attempts=10, initial_delay=5.0)

if HAS_ADK:
    input_preprocessor = Agent(
        name="InputPreprocessor",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="Parse the user input and extract: problem_title, concepts, difficulty, languages. Return ONLY JSON.",
        output_key="parsed_input",
    )

    story_agent = Agent(
        name="StoryAgent",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="Write a story (3–4 sentences) explaining the DSA problem.",
        output_key="story",
    )

    analogy_agent = Agent(
        name="AnalogyAgent",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="Write a 2–3 sentence analogy explaining the problem.",
        output_key="analogy",
    )

    tip_agent = Agent(
        name="TipAgent",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="Give one actionable tip for mastering this problem.",
        output_key="learning_tip",
    )

    # NOTE: Using SequentialAgent instead of ParallelAgent to strictly avoid 
    # "429 Resource Exhausted" errors on the free tier.
    content_team = SequentialAgent(
        name="ContentTeam",
        sub_agents=[story_agent, analogy_agent, tip_agent],
    )

    def make_code_agent(lang: str, ext: str):
        return Agent(
            name=f"CodeAgent_{lang}",
            model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
            instruction=f"Generate a beginner-friendly solution for this DSA problem in {lang}. Return ONLY code in triple backticks.",
            output_key=f"code_{lang.lower()}",
        )

    python_agent = make_code_agent("python", "python")
    c_agent = make_code_agent("c", "c")
    cpp_agent = make_code_agent("cpp", "cpp")
    java_agent = make_code_agent("java", "java")

    # NOTE: Using SequentialAgent for code generation as well to avoid rate limits.
    code_team = SequentialAgent(
        name="CodeTeam",
        sub_agents=[python_agent, c_agent, cpp_agent, java_agent],
    )

    final_aggregator = Agent(
        name="FinalAggregator",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="Return EXACT JSON with keys: story, analogy, learning_tip, code (dict).",
        output_key="final_output",
    )

    root_agent = SequentialAgent(
        name="DSAQuestFull",
        sub_agents=[
            input_preprocessor,
            content_team,
            code_team,
            final_aggregator,
        ],
    )
else:
    root_agent = None


async def run_agent(user_input: str) -> Dict[str, Any]:
    if not HAS_ADK:
        runner = InMemoryRunner(None)
        res = await runner.run_debug(user_input)
        return res.get("final_output", {})

    runner = InMemoryRunner(agent=root_agent)
    
    # Capture stdout since the agents print their output
    import sys
    from io import StringIO
    
    old_stdout = sys.stdout
    sys.stdout = captured_output = StringIO()
    
    try:
        # Run the agent - this prints output but returns empty list
        result = await runner.run_debug(user_input)
        
        # Restore stdout
        sys.stdout = old_stdout
        
        # Get the captured text
        output_text = captured_output.getvalue()
        
        print(f"DEBUG: Captured {len(output_text)} characters of output")
        
        # Look for the FinalAggregator output in the captured text
        if "FinalAggregator >" in output_text:
            # Extract everything after "FinalAggregator >"
            parts = output_text.split("FinalAggregator >")
            if len(parts) > 1:
                json_text = parts[-1].strip()
                
                # Clean up the JSON (remove markdown code fences if present)
                json_text = json_text.replace("```json", "").replace("```", "").strip()
                
                # Find the JSON object (starts with { and ends with })
                start_idx = json_text.find("{")
                if start_idx != -1:
                    # Find the matching closing brace
                    brace_count = 0
                    end_idx = -1
                    for i in range(start_idx, len(json_text)):
                        if json_text[i] == "{":
                            brace_count += 1
                        elif json_text[i] == "}":
                            brace_count -= 1
                            if brace_count == 0:
                                end_idx = i + 1
                                break
                    
                    if end_idx != -1:
                        json_str = json_text[start_idx:end_idx]
                        try:
                            parsed = json.loads(json_str)
                            print(f"DEBUG: Successfully parsed JSON from captured output")
                            return parsed
                        except Exception as e:
                            print(f"DEBUG: Failed to parse JSON: {e}")
                            print(f"DEBUG: JSON string was: {json_str[:200]}...")
                            return {"error": f"Failed to parse JSON: {str(e)}", "raw": json_str[:500]}
        
        # Fallback: check if result has anything useful
        print(f"DEBUG: run_debug returned type: {type(result)}")
        if isinstance(result, list) and len(result) > 0:
            print(f"DEBUG: Result list has {len(result)} items")
            return result[-1] if isinstance(result[-1], dict) else {"error": "Unexpected result format"}
        
        # Last resort: return the captured output as raw text
        print(f"DEBUG: Returning error with captured output sample")
        return {
            "error": "Could not extract JSON from output",
            "raw_output_sample": output_text[-500:] if len(output_text) > 500 else output_text
        }
        
    except Exception as e:
        sys.stdout = old_stdout
        print(f"Error in run_agent: {e}")
        traceback.print_exc()
        return {"error": str(e)}



Writing dsaquest/main_dsa_agents.py


In [9]:
%%writefile app.py
import gradio as gr
import asyncio
import json
from dsaquest.main_dsa_agents import run_agent
from backend_user import sign_in, sign_up, sign_out, get_current_user
from progress_tracker import get_progress_tracker
from dsa_data import STRIVER_SHEET, TOTAL_PROBLEMS
from games import (
    REARRANGE_QUESTIONS, QUIZ_QUESTIONS, FILL_BLANK_QUESTIONS,
    get_rearrange_question, check_rearrange_answer,
    get_quiz_question, get_fill_blank_question
)

# --- Helper Functions ---
def get_user_stats():
    user = get_current_user()
    if user:
        tracker = get_progress_tracker()
        return f"""
        ### 👤 {user['username']}
        **XP:** {tracker.total_xp} 💎 | **Streak:** {tracker.current_streak} 🔥
        **Solved:** {len(tracker.completed_problems)}/{TOTAL_PROBLEMS}
        """
    return "Please Sign In"

def refresh_auth_ui():
    user = get_current_user()
    if user:
        return (
            gr.update(visible=False), # Login Col
            gr.update(visible=True),  # Dashboard Col
            get_user_stats()
        )
    else:
        return (
            gr.update(visible=True),  # Login Col
            gr.update(visible=False), # Dashboard Col
            ""
        )

# --- Auth Actions ---
def login(email, password):
    res = sign_in(email, password)
    if res.error:
        return gr.update(value=res.error, visible=True), *refresh_auth_ui()
    return gr.update(visible=False), *refresh_auth_ui()

def signup(email, password, username):
    res = sign_up(email, password, username)
    if res.error:
        return gr.update(value=res.error, visible=True), *refresh_auth_ui()
    sign_in(email, password)
    return gr.update(visible=False), *refresh_auth_ui()

def logout():
    sign_out()
    return *refresh_auth_ui(),

# --- Problem Browser Actions ---
def get_step_choices():
    return [f"{k}: {v['name']}" for k, v in STRIVER_SHEET.items()]

def update_problem_dropdown(step_selection):
    if not step_selection:
        return gr.update(choices=[])
    step_key = step_selection.split(":")[0]
    problems = STRIVER_SHEET[step_key]["problems"]
    return gr.update(choices=[f"{p['title']} ({p['difficulty']})" for p in problems])

def display_problem(step_selection, problem_selection):
    if not step_selection or not problem_selection:
        return "", "", "", gr.update(visible=False)
    
    step_key = step_selection.split(":")[0]
    problems = STRIVER_SHEET[step_key]["problems"]
    
    problem = next((p for p in problems if f"{p['title']} ({p['difficulty']})" == problem_selection), None)
    
    if not problem:
        return "Problem not found", "", "", gr.update(visible=False)

    details = f"""
    # {problem['title']}
    **Difficulty:** {problem['difficulty']}
    **Concepts:** {', '.join(problem['concepts'])}
    [View Problem Link]({problem['link']})
    """
    return details, problem['id'], step_key, gr.update(visible=True)

async def generate_ai_content(problem_id, step_key):
    # Retrieve problem details
    problem = None
    step = STRIVER_SHEET.get(step_key)
    if step:
        for p in step["problems"]:
            if p["id"] == problem_id:
                problem = p
                break
    
    if not problem:
        return "Error: Problem not found."

    # Construct user input for the ADK agent
    user_input = f"""
problem_title: {problem['title']}
concepts: {', '.join(problem['concepts'])}
difficulty: {problem['difficulty']}
languages: Python, Java, C++, C
"""
    
    try:
        # Call the ADK agent asynchronously
        result = await run_agent(user_input)
        
        if isinstance(result, dict):
            print(f"DEBUG: App received keys: {list(result.keys())}")
            if "error" in result:
                return f"⚠️ **AI Agent Error:** {result['error']}\n\n*This usually means the API rate limit was exceeded. Please wait a few seconds and try again.*"
                
            story = result.get("story", "No story generated.")
            analogy = result.get("analogy", "No analogy generated.")
            tip = result.get("learning_tip", "No tip generated.")
            code = result.get("code", {})
            
            md = f"## 📖 Story\n{story}\n\n## 💡 Analogy\n{analogy}\n\n## 🧠 Learning Tip\n{tip}\n\n## 💻 Generated Code"
            
            if isinstance(code, dict):
                for lang, snippet in code.items():
                    md += f"\n**{lang.capitalize()}**\n```\n{snippet}\n```\n"
            else:
                md += f"\n{code}"
                
            return md
        else:
            return f"Raw Output: {result}"
            
    except Exception as e:
        return f"Error generating content: {str(e)}"

def mark_complete(problem_id, step_key):
    tracker = get_progress_tracker()
    if not tracker:
        return "Please Sign In first."
    
    step = STRIVER_SHEET.get(step_key)
    xp = step['xp'] if step else 10
    
    is_new, new_badges = tracker.mark_problem_complete(problem_id, xp)
    msg = f"✅ Marked as complete! +{xp} XP."
    if new_badges:
        msg += f"\n🏆 New Badges: {', '.join(new_badges)}"
    return msg

# --- Game Actions ---
def load_rearrange_ui(idx):
    q, shuffled = get_rearrange_question(idx)
    # Show lines with letters (A, B, C) instead of numbers
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    lines_text = "\n".join([f"{letters[i]}) {line}" for i, (_, line) in enumerate(shuffled)])
    return (
        f"### {q['title']}\n**Language:** {q['language']}\n\nRearrange these lines in the correct order:\n\n{lines_text}",
        ""
    )

def check_rearrange_ui(idx, user_input):
    q, shuffled = get_rearrange_question(idx)
    try:
        # Convert letter input (A, B, C) to indices
        letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
        user_input = user_input.upper().replace(" ", "")
        indices = []
        for char in user_input.split(","):
            char = char.strip()
            if char in letters:
                # Get the original line number from shuffled list
                idx_in_shuffled = letters.index(char)
                original_num = shuffled[idx_in_shuffled][0]
                indices.append(original_num)
    except:
        return "❌ Invalid format. Use comma-separated letters (e.g., B, A, C)."
    
    if check_rearrange_answer(q, indices):
        return "✅ Correct!"
    else:
        return "❌ Incorrect. Try again."

def load_quiz_ui(idx):
    q = get_quiz_question(idx)
    return q['question'], gr.update(choices=q['options'], value=None), ""

def check_quiz_ui(idx, answer):
    q = get_quiz_question(idx)
    correct = q['options'][q['answer_index']]
    if answer == correct:
        return f"✅ Correct!\n\n{q.get('explanation','')}"
    else:
        return f"❌ Incorrect. The correct answer was: **{correct}**\n\n{q.get('explanation','')}"

def load_fill_blank_ui(idx):
    q = get_fill_blank_question(idx)
    return (
        f"### Fill in the Blank\n**Language:** {q['language']}\n\n```{q['language'].lower()}\n{q['snippet']}\n```\n\n**Hint:** {q['hint']}",
        ""
    )

def check_fill_blank_ui(idx, user_answer):
    q = get_fill_blank_question(idx)
    if user_answer.strip().lower() == q['answer'].lower():
        return "✅ Correct!"
    else:
        return f"❌ Incorrect. The correct answer was: **{q['answer']}**"

# --- Main UI Construction ---
with gr.Blocks(title="DSA Quest", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚀 DSA Quest - Master Coding with Fun!")
    
    # Global User State
    # Note: In a real multi-user web app, we'd use request.session.
    # For a notebook demo, we rely on the backend_user file.
    
    with gr.Tab("🏠 Home"):
        with gr.Row():
            with gr.Column(visible=True) as login_col:
                gr.Markdown("### Sign In / Sign Up")
                auth_msg = gr.Markdown(visible=False)
                with gr.Tabs():
                    with gr.Tab("Sign In"):
                        si_email = gr.Textbox(label="Email")
                        si_pass = gr.Textbox(label="Password", type="password")
                        si_btn = gr.Button("Sign In", variant="primary")
                    with gr.Tab("Sign Up"):
                        su_email = gr.Textbox(label="Email")
                        su_pass = gr.Textbox(label="Password", type="password")
                        su_user = gr.Textbox(label="Username")
                        su_btn = gr.Button("Create Account", variant="primary")
            
            with gr.Column(visible=False) as dashboard_col:
                user_stats_md = gr.Markdown()
                logout_btn = gr.Button("Sign Out")

    with gr.Tab("📖 Problem Browser"):
        with gr.Row():
            with gr.Column(scale=1):
                step_dd = gr.Dropdown(choices=get_step_choices(), label="Select Topic")
                prob_dd = gr.Dropdown(choices=[], label="Select Problem")
            with gr.Column(scale=2):
                prob_details_md = gr.Markdown("Select a problem to view details.")
                # Hidden state
                curr_prob_id = gr.Textbox(visible=False)
                curr_step_key = gr.Textbox(visible=False)
                
                with gr.Row(visible=False) as action_row:
                    gen_ai_btn = gr.Button("✨ Generate AI Content (Story, Analogy, Code)", variant="primary")
                    complete_btn = gr.Button("✅ Mark Complete")
                
                ai_output_md = gr.Markdown()
                status_output_md = gr.Markdown()

    with gr.Tab("🎮 Coding Games"):
        with gr.Tabs():
            with gr.Tab("🧩 Rearrange Code"):
                rearr_idx = gr.State(0)
                rearr_q_md = gr.Markdown()
                rearr_input = gr.Textbox(label="Your Order (e.g., B, A, C)", placeholder="Enter letters separated by commas")
                rearr_check_btn = gr.Button("Check Answer")
                rearr_res_md = gr.Markdown()
                rearr_next_btn = gr.Button("Next Puzzle")
            
            with gr.Tab("📝 Fill in the Blanks"):
                fill_idx = gr.State(0)
                fill_q_md = gr.Markdown()
                fill_input = gr.Textbox(label="Your Answer")
                fill_check_btn = gr.Button("Check Answer")
                fill_res_md = gr.Markdown()
                fill_next_btn = gr.Button("Next Question")
            
            with gr.Tab("🃏 Quiz"):
                quiz_idx = gr.State(0)
                quiz_q_md = gr.Markdown()
                quiz_opts = gr.Radio(label="Options")
                quiz_check_btn = gr.Button("Check Answer")
                quiz_res_md = gr.Markdown()
                quiz_next_btn = gr.Button("Next Question")

    # --- Event Wiring ---
    
    # Auth
    si_btn.click(login, [si_email, si_pass], [auth_msg, login_col, dashboard_col, user_stats_md])
    su_btn.click(signup, [su_email, su_pass, su_user], [auth_msg, login_col, dashboard_col, user_stats_md])
    logout_btn.click(logout, [], [login_col, dashboard_col, user_stats_md])
    
    # Problem Browser
    step_dd.change(update_problem_dropdown, step_dd, prob_dd)
    prob_dd.change(display_problem, [step_dd, prob_dd], [prob_details_md, curr_prob_id, curr_step_key, action_row])
    
    # AI Generation (Async)
    gen_ai_btn.click(generate_ai_content, [curr_prob_id, curr_step_key], ai_output_md)
    
    complete_btn.click(mark_complete, [curr_prob_id, curr_step_key], status_output_md)
    
    # Games - Rearrange
    demo.load(load_rearrange_ui, [rearr_idx], [rearr_q_md, rearr_res_md])
    rearr_check_btn.click(check_rearrange_ui, [rearr_idx, rearr_input], rearr_res_md)
    def next_rearr(i): return (i + 1) % len(REARRANGE_QUESTIONS)
    rearr_next_btn.click(next_rearr, rearr_idx, rearr_idx).then(
        load_rearrange_ui, [rearr_idx], [rearr_q_md, rearr_res_md]
    )
    
    # Games - Fill in the Blanks
    demo.load(load_fill_blank_ui, [fill_idx], [fill_q_md, fill_res_md])
    fill_check_btn.click(check_fill_blank_ui, [fill_idx, fill_input], fill_res_md)
    def next_fill(i): return (i + 1) % len(FILL_BLANK_QUESTIONS)
    fill_next_btn.click(next_fill, fill_idx, fill_idx).then(
        load_fill_blank_ui, [fill_idx], [fill_q_md, fill_res_md]
    )
    
    # Games - Quiz
    demo.load(load_quiz_ui, [quiz_idx], [quiz_q_md, quiz_opts, quiz_res_md])
    quiz_check_btn.click(check_quiz_ui, [quiz_idx, quiz_opts], quiz_res_md)
    def next_quiz(i): return (i + 1) % len(QUIZ_QUESTIONS)
    quiz_next_btn.click(next_quiz, quiz_idx, quiz_idx).then(
        load_quiz_ui, [quiz_idx], [quiz_q_md, quiz_opts, quiz_res_md]
    )

    # Init Auth Check
    demo.load(refresh_auth_ui, [], [login_col, dashboard_col, user_stats_md])

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", share=True)


Writing app.py


In [ ]:
!python app.py

Gemini API key loaded.
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://4581d7cad5c4a79afd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
DEBUG: Captured 20819 characters of output
DEBUG: Successfully parsed JSON from captured output
DEBUG: App received keys: ['story', 'analogy', 'learning_tip', 'code']
